In [1]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np

In [2]:
uri = "bolt://34.101.192.24:7687"
username = "neo4j"
password = "unej1234"

driver = GraphDatabase.driver(uri, auth=(username, password))

In [23]:
def extract_node_properties_and_relationships():
    with driver.session() as session:
        # Cypher query to fetch node properties and relationships
        query = """
        MATCH (n)-[r]->(m)
            WHERE any(label IN labels(n) WHERE label IN ['Gejala', 'PenyakitPadi', 'HamaPadi']) 
            AND any(label IN labels(m) WHERE label IN ['Gejala', 'PenyakitPadi', 'HamaPadi'])
        RETURN 
        n.Vector AS source_vector, n.label AS source_label, labels(n) AS source_class, 
        type(r) AS relationship_type,
        m.Vector AS target_vector, m.label AS target_label, labels(m) AS target_class

        """
        result = session.run(query)
        # Extract properties and relationships and store in DataFrame
        df = pd.DataFrame([record.values() for record in result], columns=result.keys())
        # Clean labels
        df['source_class'] = df['source_class'].apply(lambda entry: [item for item in entry if item not in ["Resource", "NamedIndividual"]])
        df['target_class'] = df['target_class'].apply(lambda entry: [item for item in entry if item not in ["Resource", "NamedIndividual"]])
        return df

# Call the function to extract node properties and relationships
Rice_KG_df = extract_node_properties_and_relationships()

# Print the DataFrame
Rice_KG_df


,source_vector,source_label,source_class,relationship_type,target_vector,target_label,target_class
0,"[0.03452454134821892, 0.04011273384094238, -0....",Gosong bulir,[PenyakitPadi],memilikiGejala,"[0.03818255290389061, -0.015258912928402424, -...",Bulir terdapat bercak,[Gejala]
1,"[0.03452454134821892, 0.04011273384094238, -0....",Gosong bulir,[PenyakitPadi],memilikiGejala,"[0.03753756731748581, 0.005460740067064762, -0...",Bulir pecah,[Gejala]
2,"[0.03452454134821892, 0.04011273384094238, -0....",Gosong bulir,[PenyakitPadi],memilikiGejala,"[0.03612148016691208, 0.0048484764993190765, 0...",Bulir berubah warna,[Gejala]
3,"[0.03452454134821892, 0.04011273384094238, -0....",Gosong bulir,[PenyakitPadi],memilikiGejala,"[0.03656195476651192, 0.02377898246049881, 0.0...",Bulir mengalami kerusakan,[Gejala]
4,"[0.03643496334552765, -0.020186755806207657, -...",Garis Merah,[PenyakitPadi],memilikiGejala,"[0.038021307438611984, -0.010079000145196915, ...",Malai terdapat bercak,[Gejala]
...,...,...,...,...,...,...,...
131,"[0.03919482231140137, -0.031110592186450958, 0...",Penggerek Batang Kuning,[HamaPadi],memilikiGejala,"[0.038021307438611984, -0.010079000145196915, ...",Malai terdapat bercak,[Gejala]
132,"[0.03919482231140137, -0.031110592186450958, 0...",Penggerek Batang Kuning,[HamaPadi],memilikiGejala,"[0.033894460648298264, -0.014245453290641308, ...",Daun terdapat bercak,[Gejala]
133,"[0.03914107382297516, -0.029383953660726547, -...",Rayap,[HamaPadi],memilikiGejala,"[0.03758235648274422, 0.02068854123353958, 0.0...",Batang rapuh,[Gejala]
134,"[0.03914107382297516, -0.029383953660726547, -...",Rayap,[HamaPadi],memilikiGejala,"[0.03786005824804306, -0.004899086430668831, 0...",Akar berlubang,[Gejala]


In [25]:
# Rice_KG_df.to_csv("Rice KG.csv", index=False)

In [34]:
import numpy as np
import pandas as pd

# Function to generate random vectors of the same length as the original vectors
def generate_random_vector(length):
    return list(np.random.uniform(low=-0.05, high=0.05, size=length))

# Assuming Rice_KG_df is your original DataFrame

# Convert list-like columns to strings or other hashable types
Rice_KG_df['source_class'] = Rice_KG_df['source_class'].apply(lambda x: str(x))
Rice_KG_df['target_class'] = Rice_KG_df['target_class'].apply(lambda x: str(x))

# Extract unique values from the existing dataset
unique_source_labels = Rice_KG_df['source_label'].unique()
unique_source_classes = Rice_KG_df['source_class'].unique()
unique_relationship_types = Rice_KG_df['relationship_type'].unique()
unique_target_labels = Rice_KG_df['target_label'].unique()
unique_target_classes = Rice_KG_df['target_class'].unique()

# Length of the vectors in the original dataset (assuming all vectors are of the same length)
vector_length = len(Rice_KG_df['source_vector'][0])  # Adjust based on your actual data structure

# Number of synthetic entries to generate
num_synthetic_entries = 10000

# Generate synthetic data
synthetic_data = {
    'source_vector': [
        generate_random_vector(vector_length) if label not in unique_source_labels
        else Rice_KG_df[Rice_KG_df['source_label'] == label]['source_vector'].iloc[0]
        for label in np.random.choice(unique_source_labels, num_synthetic_entries)
    ],
    'source_label': np.random.choice(unique_source_labels, num_synthetic_entries),
    'source_class': [
        np.random.choice(Rice_KG_df[Rice_KG_df['source_label'] == label]['source_class'].unique())
        if label in unique_source_labels
        else np.random.choice(unique_source_classes)
        for label in np.random.choice(unique_source_labels, num_synthetic_entries)
    ],
    'relationship_type': ['memilikiGejala'] * num_synthetic_entries,  # Set relationship_type to 'memilikiGejala'
    'target_vector': [
        generate_random_vector(vector_length) if label not in unique_target_labels
        else Rice_KG_df[Rice_KG_df['target_label'] == label]['target_vector'].iloc[0]
        for label in np.random.choice(unique_target_labels, num_synthetic_entries)
    ],
    'target_label': np.random.choice(unique_target_labels, num_synthetic_entries),
    'target_class': [
        np.random.choice(Rice_KG_df[Rice_KG_df['target_label'] == label]['target_class'].unique())
        if label in unique_target_labels
        else np.random.choice(unique_target_classes)
        for label in np.random.choice(unique_target_labels, num_synthetic_entries)
    ]
}

# Create a DataFrame for the synthetic data
synthetic_df = pd.DataFrame(synthetic_data)
synthetic_df


,source_vector,source_label,source_class,relationship_type,target_vector,target_label,target_class
0,"[0.03741215541958809, 0.04282289370894432, -0....",Bulir mengalami kerusakan,['HamaPadi'],memilikiGejala,"[0.03407501056790352, 0.028321780264377594, -0...",Batang berlubang,['Gejala']
1,"[0.03759131580591202, 0.0037341024726629257, -...",Bakanae,['PenyakitPadi'],memilikiGejala,"[0.03637301176786423, 0.009537891484797001, 0....",Bibit tumbuh tidak normal,['Gejala']
2,"[0.03915898874402046, 0.03670716658234596, -0....",Bercak Coklat,['HamaPadi'],memilikiGejala,"[0.03794068470597267, 0.04251095652580261, 0.0...",Daun menguning,['Gejala']
3,"[0.037698812782764435, 0.000280825886875391, -...",Virus Kerdil Rumput,['PenyakitPadi'],memilikiGejala,"[0.035620369017124176, 0.006964750122278929, 0...",Tanaman layu,['Gejala']
4,"[0.03778839483857155, 0.03073642961680889, -0....",Bercak Coklat,['HamaPadi'],memilikiGejala,"[0.03793172538280487, -0.0405346043407917, -0....",Lalat Tunas,['Gejala']
...,...,...,...,...,...,...,...
9995,"[0.03959793969988823, 0.005939625669270754, -0...",Phoma Sorghina,['HamaPadi'],memilikiGejala,"[0.034550994634628296, 0.011215672828257084, 0...",Malai basah,['Gejala']
9996,"[0.03452454134821892, 0.04011273384094238, -0....",Kutu Putih,['PenyakitPadi'],memilikiGejala,"[0.03955314680933952, -0.009288176894187927, -...",Tanaman mengeluarkan cairan,['Gejala']
9997,"[0.0374569445848465, -0.041949301958084106, 0....",Hawar Daun Bakteri,['HamaPadi'],memilikiGejala,"[0.03786005824804306, -0.004899086430668831, 0...",Malai hilang,['PenyakitPadi']
9998,"[0.03948148339986801, 0.026347341015934944, 0....",Penggerek Batang Ungu,['PenyakitPadi'],memilikiGejala,"[0.0393919013440609, -0.004108262713998556, -0...",Pelepah membusuk,['Gejala']


In [37]:
import pandas as pd

# Duplicate the data by appending it to itself, for example, to create synthetic data
synthetic_data = pd.concat([Rice_KG_df] * 250, ignore_index=True)  # Adjust the multiplier (2 in this case) as needed

# Save the synthetic data to a new CSV file
synthetic_data.to_csv('synthetic_data.csv', index=False)

print("Synthetic data generated successfully and saved to 'synthetic_data.csv'.")


Synthetic data generated successfully and saved to 'synthetic_data.csv'.


In [33]:
# synthetic_df.to_csv("Syntethic Rice KG.csv", index=False)